# Heroes of Pymoli - Solution
### by: Richard Guarnieri

In [1]:
# Import Dependencies
import pandas as pd
import numpy as np

In [2]:
# Create file path, read and store into Pandas DataFrame
file_path = "Resources/purchase_data.csv"
purchase_df = pd.read_csv(file_path)

# Display DataFrame
purchase_df.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


### Player Count
* Display the total number of players

In [3]:
# Drop duplicates from subset (column) SN
uniques_df = purchase_df.drop_duplicates(subset='SN')

# Create the total_players DataFrame
total_players_df = pd.DataFrame({
    'Total Players': [len(uniques_df)]
})

# Display DataFrame
total_players_df

### Another solution
### Count unique values in 'SN' column
### len(purchase_df['SN'].value_counts())

,Total Players
0,576


### Purchasing Analysis (Total)
* Run basic calculations to obtain number of unique items, average price, etc.
* Create a summary data frame to hold the results
* Optional: give the displayed data cleaner formatting
* Display the summary data frame

In [4]:
# Create variables and run calculations
unique_items = len(purchase_df['Item Name'].value_counts())
average_price = purchase_df['Price'].mean()
number_purchases = len(purchase_df)
total_revenue = purchase_df['Price'].sum()

# Create the summary DataFrame
summary_df = pd.DataFrame({
    '# of Unique Items': [unique_items],
    'Average Price': [average_price],
    '# of Purchases': [number_purchases],
    'Total Revenue': [total_revenue]
})

# Apply number formatting
summary_df[['Average Price', 'Total Revenue']] = summary_df[['Average Price', 'Total Revenue']].applymap("${:,.2f}".format)

# Display DataFrame
summary_df

### Another way to apply number formatting using f-strings
### Summary_df[['Average Price', 'Total Revenue']] = summary_df[['Average Price', 'Total Revenue']].applymap(lambda x: f'{x:,.2f}')

,# of Unique Items,Average Price,# of Purchases,Total Revenue
0,179,$3.05,780,"$2,379.77"


### Gender Demographics
* Percentage and Count of Male Players
* Percentage and Count of Female Players
* Percentage and Count of Other / Non-Disclosed

In [5]:
# Create a new DataFrame based on 'uniques_df' grouped by Gender and aggregate by count based on 'SN' column
gender_df = uniques_df.groupby('Gender')['SN'].agg([('# of Players', 'count')])

# Add Percentage of Players column and apply percentage format
gender_df['Percetange of Players'] = (gender_df['# of Players']/gender_df['# of Players'].sum()).map(lambda x: f'{x:,.2%}')

# Remove index name and display sorted DataFrame
gender_df.rename_axis(None, inplace=True)
gender_df.sort_values('# of Players', ascending=False)

,# of Players,Percetange of Players
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%



### Purchasing Analysis (Gender)
* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender
* Create a summary data frame to hold the results
* Optional: give the displayed data cleaner formatting
* Display the summary data frame

In [6]:
# Create a new DataFrame based on 'purchase_df' grouped by Price and aggregate by count, mean and sum
purchase_analysis = purchase_df.groupby('Gender')['Price'].agg([
    ('Purchase Count', 'count'), 
    ('Average Purchase Price', 'mean'),
    ('Total Purchase Value','sum')
])

# Remove index name
purchase_analysis.rename_axis(None, inplace=True)

# Merge with gender_df
purchase_analysis = purchase_analysis.merge(gender_df['# of Players'], left_index=True, right_index=True)

# Create Avg Total Purchase per Person colum
purchase_analysis['Avg Total Purchase per Person'] = purchase_analysis['Total Purchase Value']/purchase_analysis['# of Players']

# Delete # of Players column
del purchase_analysis['# of Players']

# Format columns
purchase_analysis.iloc[:,1:] = purchase_analysis.iloc[:,1:].applymap('${:,.2f}'.format)

# Display sorted DataFrame
purchase_analysis.sort_values('Purchase Count', ascending=False)

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Male,652,$3.02,"$1,967.64",$4.07
Female,113,$3.20,$361.94,$4.47
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


### Age Demographics
* Establish bins for ages
* Categorize the existing players using the age bins. Hint: use pd.cut()
* Calculate the numbers and percentages by age group
* Create a summary data frame to hold the results
* Optional: round the percentage column to two decimal points
* Display Age Demographics Table

In [7]:
# Create a new DataFrame via copy() method
age_demo_df = uniques_df.copy()

# Create the bins in which Data will be held
bins = [0, 9, 14, 19, 24, 29, 34, 39, 999]

# Create the labels for the bins
labels = ['<10', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39', '40+']

# Add Bins column
age_demo_df['Age Bins'] = pd.cut(age_demo_df['Age'], bins, labels=labels, include_lowest=True)

# Groupby
age_demo_df = age_demo_df.groupby('Age Bins')['SN'].agg([('Total Count', 'count')])

# Add 'Percentage of Players' column
age_demo_df['Percentage of Players'] = (age_demo_df['Total Count']/age_demo_df['Total Count'].sum()).map('{:,.2%}'.format)

# Remove index name
age_demo_df.rename_axis(None, inplace=True)

# Display DataFrame
age_demo_df

,Total Count,Percentage of Players
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


### Purchasing Analysis (Age)
* Bin the purchase_data data frame by age
* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below
* Create a summary data frame to hold the results
* Optional: give the displayed data cleaner formatting
* Display the summary data frame

In [8]:
# Create a new DataFrame via copy() method
purchase_age = purchase_df.copy()

# Create the bins in which Data will be held
bins = [0, 9, 14, 19, 24, 29, 34, 39, 999]

# Create the labels for the bins
labels = ['<10', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39', '40+']

# Add Bins column
purchase_age['Age Bins'] = pd.cut(purchase_age['Age'], bins, labels=labels, include_lowest=True)

# Groupby
purchase_age = purchase_age.groupby('Age Bins')['Price'].agg([
    ('Purchase Count', 'count'),
    ('Average Purchase Price', 'mean'),
    ('Total Purchase Value', 'sum')
])

# Remove index name
purchase_age.rename_axis(None, inplace=True)

# Merge with age_demo_df
purchase_age = purchase_age.merge(age_demo_df['Total Count'], left_index=True, right_index=True)

# Do calcuations to get the 'Avg Total Purchase per Person'
purchase_age['Avg Total Purchase per Person'] = purchase_age['Total Purchase Value']/purchase_age['Total Count']

# Remove 'Total Count' column
del purchase_age['Total Count']

# Apply formatting
purchase_age.iloc[:,1:] = purchase_age.iloc[:,1:].applymap('${:,.2f}'.format)

# Diplay DataFrame
purchase_age

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,"$1,114.06",$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


### Top Spenders
* Run basic calculations to obtain the results in the table below
* Create a summary data frame to hold the results
* Sort the total purchase value column in descending order
* Optional: give the displayed data cleaner formatting
* Display a preview of the summary data frame

In [9]:
# Groupby
top_spenders = purchase_df.groupby('SN')['Price'].agg([
    ('Purchase Count', 'count'),
    ('Average Purchase Price', 'mean'),
    ('Total Purchase Value', 'sum')
])

# Sort DataFrame
top_spenders = top_spenders.sort_values('Total Purchase Value', ascending=False)[:5]

# Apply formatting
top_spenders.iloc[:,1:] = top_spenders.iloc[:,1:].applymap('${:,.2f}'.format)

# Diplay DataFrame
top_spenders

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


### Most Popular Items
* Retrieve the Item ID, Item Name, and Item Price columns
* Group by Item ID and Item Name. Perform calculations to obtain purchase count, average item price, and total purchase value
* Create a summary data frame to hold the results
* Sort the purchase count column in descending order
* Optional: give the displayed data cleaner formatting
* Display a preview of the summary data frame

In [10]:
# Groupby
popular_items = purchase_df.groupby(['Item ID', 'Item Name'])['Price'].agg([
    ('Purchase Count', 'count'),
    ('Average Purchase Price', 'mean'),
    ('Total Purchase Value', 'sum')
])

# Sort DataFrame
popular_items = popular_items.sort_values('Purchase Count', ascending=False)[:5]

# Apply formatting
popular_items.iloc[:,1:] = popular_items.iloc[:,1:].applymap('${:,.2f}'.format)

# Diplay DataFrame
popular_items

,,Purchase Count,Average Purchase Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
132,Persuasion,9,$3.22,$28.99
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77


### Most Profitable Items
* Sort the above table by total purchase value in descending order
* Optional: give the displayed data cleaner formatting
* Display a preview of the data frame

In [11]:
# Groupby
profitable_items = purchase_df.groupby(['Item ID', 'Item Name'])['Price'].agg([
    ('Purchase Count', 'count'),
    ('Average Purchase Price', 'mean'),
    ('Total Purchase Value', 'sum')
])

# Sort DataFrame
profitable_items = profitable_items.sort_values('Total Purchase Value', ascending=False)[:5]

# Apply formatting
profitable_items.iloc[:,1:] = profitable_items.iloc[:,1:].applymap('${:,.2f}'.format)

# Diplay DataFrame
profitable_items

,,Purchase Count,Average Purchase Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
103,Singed Scalpel,8,$4.35,$34.80


### Observable trends based on the data:
* 63% of the player base is between 15 and 24 years old
* Male players outweights Females by significant margin - over 84% player base is Male.
* Final Critic although having a high price, is the most popular item.